# Create Elastic search local

### Notes

Links:
-  url

### import

In [1]:
# imports
import os
import json

import pandas as pd
import numpy as np

from elasticsearch import Elasticsearch
from tqdm import tqdm_notebook

### functions

In [2]:
def code_output(cell,temp_dict):
    """
    Explanation function
    """
    if cell['outputs']!=[]:
        output_type = cell['outputs'][0]['output_type']
        temp_dict['output_type'] = output_type
        if output_type == 'stream':
            temp_dict['output'] = cell['outputs'][0]['text']
        elif output_type == 'error':
            temp_dict['output'] = cell['outputs'][0]['traceback']
        elif temp_dict['output_type'] == 'display_data':
            temp_dict['output'] = 'displayed data'
        elif 'data' in cell['outputs'][0].keys():
            temp_dict['output'] = list(cell['outputs'][0]['data'].values())
        elif 'text' in cell['outputs'][0].keys():
            temp_dict['output'] = cell['outputs'][0]['text']
        elif 'ename' in cell['outputs'][0].keys():
            temp_dict['output'] = cell['outputs'][0]['ename']+cell['outputs'][0]['evalue']
        else:
            temp_dict['output'] = 'unknown'
            print(cell)
    else:
        temp_dict['output'] = 'empty'
    return temp_dict


def read_ipynb_cell(cell_id,cell_dict,file,folder,location,repo):
    """
    Explanation function
    """
    with open(location,encoding="utf8") as notebook:
        data = json.load(notebook)
        file_cell = 0
        nbformat = data['nbformat']
#         print(nbformat,'- file_name',location)
        
        if nbformat == 4: # current nbformat
            data_cells =  data['cells']
        elif nbformat == 3: # old nbformat
            data_cells =  data['worksheets'][0]['cells']

        for cell in data_cells:
            temp_dict = {}
            if cell['cell_type'] == 'code' and nbformat == 3: #cell['source'] doesn't exist within this condition, use cell['input']
                text = cell['input']
            else:
                text = cell['source']
            clean_cell = list(map(lambda s: s.strip(), text)) #remove the '\n' at the end of each string in the list         
            single_string = ' '.join(clean_cell)
            lines = len(clean_cell)

            temp_dict['file_cell'] = file_cell
            temp_dict['file'] = file
            temp_dict['nbformat'] = data['nbformat']
            temp_dict['folder'] = folder
            temp_dict['repo'] =  repo
            temp_dict['location'] = location
            temp_dict['string'] = clean_cell
#             temp_dict['char'] = single_string
            temp_dict['lines'] = lines
            temp_dict['cell_type'] = cell['cell_type']
            if cell['cell_type'] == 'code':
                temp_dict = code_output(cell,temp_dict)

            cell_dict[cell_id] = temp_dict
            
            cell_id += 1
            file_cell += 1 
    return cell_id,cell_dict


def files_to_dict(file_dict):
    """
    Explanation function
    """
    cell_id = 0
    cell_dict = {}

    for file in file_dict.keys():
        try:
            file_name = file_dict[file]['file']
            folder = file_dict[file]['folder']
            location = file_dict[file]['location']
            repo = file_dict[file]['repo']
            #kan ik dit niet in één regel schrijven, ff controleren nog bijv a,b,c = dict.values()
            
            cell_id_dict = read_ipynb_cell(cell_id,cell_dict,file_name,folder,location,repo)
            cell_id = cell_id_dict[0]
            cell_dict = cell_id_dict[1]
        except Exception as e:
#             print(e)
            print("failed for file:",cell_dict[file]['file'])

        
    return cell_id,cell_dict


def rec_to_actions(df):
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))
      

def index_marks(nrows, chunk_size):
    return range(1 * chunk_size, (nrows // chunk_size + 1) * chunk_size, chunk_size)


def split(dfm, chunk_size):
    indices = index_marks(dfm.shape[0], chunk_size)
    return np.split(dfm, indices) 
    

def create_SE_from_folder(repo_name,es,folder,file_id):
    """
    Explenation
    """
    cwd = os.getcwd()
    if folder == 'single_files':
        path = cwd+"\\"+folder
    elif folder == 'notebooks1':
        path = 'E:\\Files\\Universiteit\\thesis\\notebooks1'
    elif folder == 'notebooks':
        path = cwd+"\\"+folder
    else:
        path = cwd+"\\"+folder+"\\"+repo_name
#         print(path)
    # CREATE DICT FOR ALL 'IPYNB' FILES
#     file_id = 0
    file_dict = {}
    
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".ipynb"):
                temp_dict = {}
                temp_dict['file'] = file
                temp_dict['folder'] = root.split('\\')[-1]
                temp_dict['location'] = os.path.join(root,file)
                temp_dict['repo'] = repo_name

                file_dict[file_id] = temp_dict
                file_id += 1
    
    # CREATE DICT FOR ALL CELLS
    cell_dict = files_to_dict(file_dict)[1]
    
    # CREATE DATAFRAME FROM DICT
    cell_df = pd.DataFrame.from_dict(cell_dict,orient='index')
    cell_df.index = cell_df.index.set_names(['cell_id'])
    cell_df = cell_df.fillna('empty').reset_index()
    return cell_df
    # PUT DATAFRAME INTO ELASTIC SEARCH
    
#     r = es.bulk(rec_to_actions(cell_df))
    for chuck in split(cell_df, 50000):
        r = es.bulk(rec_to_actions(chuck))
    return es,file_id,cell_df
#     return cell_df,es,file_id

### Create local elastic search variable

#### Elastic Search curl's

In [4]:
# !curl "http://localhost:9200/test"
# !curl -XDELETE "localhost:9200/test_df_extra"
# !curl -XPOST "http://localhost:9200/_shutdown"

!curl "http://localhost:9200/_cat/indices?v"

health status index         uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   restart       Gn2kwcjvRLehQoUX6WrTyQ   1   1     176812            0    916.6mb        916.6mb
yellow open   gallery_repos kNzSILR7T6Wp4RX1Cn9XaQ   1   1      42437            0     90.2mb         90.2mb
yellow open   gallery       x4vbbCLoT1aMRkZZLvVbOA   1   1       5654            0      7.6mb          7.6mb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   436  100   436    0     0   2037      0 --:--:-- --:--:-- --:--:--  2037


### Count .ipynb files in current repository

In [5]:
def count_ipynb(folder):
    path = os.getcwd()+'\\'+folder
    try:
        dir_list = os.listdir(path)
    except:
        return "this repository doesn't exist"

    file_id = 0
    for root, dirs, files in os.walk(path):
#         for file in tqdm_notebook(files):
        for file in files:
            if file.endswith(".ipynb"):
                file_id += 1
    return file_id

count_ipynb('single_files'),count_ipynb('repos'),count_ipynb('notebooks'),count_ipynb('fake_folder')

(87, 1201, 6529, "this repository doesn't exist")

### Put single_files into elastic search

In [17]:
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST]) 


INDEX = "gallery"
TYPE = "record"

In [26]:
gallery_single_df = create_SE_from_folder('unkown',es,'single_files',0)
gallery_single_df

,cell_id,file_cell,file,nbformat,folder,repo,location,string,lines,cell_type,output_type,output
0,0,0,%20Python_Dictionary.ipynb,4,single_files,unkown,C:\Users\kenne\Documents\thesis\single_files\%...,[],1,markdown,empty,empty
1,1,1,%20Python_Dictionary.ipynb,4,single_files,unkown,C:\Users\kenne\Documents\thesis\single_files\%...,[# Python Dictionary from Scratch!!!],1,markdown,empty,empty
2,2,2,%20Python_Dictionary.ipynb,4,single_files,unkown,C:\Users\kenne\Documents\thesis\single_files\%...,[## Let us understand the fundamentals of dict...,1,markdown,empty,empty
3,3,3,%20Python_Dictionary.ipynb,4,single_files,unkown,C:\Users\kenne\Documents\thesis\single_files\%...,[![alt text](https://www.learntek.org/blog/wp-...,1,markdown,empty,empty
4,4,4,%20Python_Dictionary.ipynb,4,single_files,unkown,C:\Users\kenne\Documents\thesis\single_files\%...,[Dictionary in python comprises an unordered c...,1,markdown,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...
5651,5651,190,numerical-slides-checkpoint.ipynb,4,.ipynb_checkpoints,unkown,C:\Users\kenne\Documents\thesis\single_files\....,"[## Discrete random numbers, , - The `randint(...",7,markdown,empty,empty
5652,5652,191,numerical-slides-checkpoint.ipynb,4,.ipynb_checkpoints,unkown,C:\Users\kenne\Documents\thesis\single_files\....,"[die_roll = randint(0, 6) + 1, die_roll]",2,code,execute_result,[[4]]
5653,5653,192,numerical-slides-checkpoint.ipynb,4,.ipynb_checkpoints,unkown,C:\Users\kenne\Documents\thesis\single_files\....,"[- Just as with the `normal()` function, we ca...",3,markdown,empty,empty
5654,5654,193,numerical-slides-checkpoint.ipynb,4,.ipynb_checkpoints,unkown,C:\Users\kenne\Documents\thesis\single_files\....,"[bernoulli_trials = randint(0, 2, size = 20), ...",2,code,execute_result,"[[array([1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1..."


In [27]:
for chuck in tqdm_notebook(split(gallery_single_df, 500)):
    try:
        r = es.bulk(rec_to_actions(chuck))
    except:
        print('failed, skip this df')

In [28]:
!curl "http://localhost:9200/_cat/indices?v"

health status index   uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   restart Gn2kwcjvRLehQoUX6WrTyQ   1   1     176812            0    916.6mb        916.6mb
yellow open   gallery x4vbbCLoT1aMRkZZLvVbOA   1   1       5654            0       230b           230b


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   309  100   309    0     0  10300      0 --:--:-- --:--:-- --:--:-- 10300


### Put github repositories into elastic search

In [57]:
folder = 'repos'
path = path = os.getcwd()+'//'+folder
dir_list = os.listdir(path)
file_id = 0
gallery_repo_df = None

for repo in tqdm_notebook(dir_list):
    try:
        df = create_SE_from_folder(repo,es,folder,file_id)

        if gallery_repo_df is None:
            gallery_repo_df = df
        else:
            gallery_repo_df = pd.concat([gallery_repo_df,df])
        file_id = 0
    except Exception as e:
        print(e,repo)

gallery_repo_df   
# cell_df,es,file_id

C:\Users\kenne\Documents\thesis\repos\2014-08-st-louis-county-segregation
C:\Users\kenne\Documents\thesis\repos\AeroPython
C:\Users\kenne\Documents\thesis\repos\algorithms_in_ipython_notebooks
C:\Users\kenne\Documents\thesis\repos\big_data
C:\Users\kenne\Documents\thesis\repos\bioinf-python
C:\Users\kenne\Documents\thesis\repos\bqplot
C:\Users\kenne\Documents\thesis\repos\codekatas
C:\Users\kenne\Documents\thesis\repos\colour
C:\Users\kenne\Documents\thesis\repos\CS1001.py
C:\Users\kenne\Documents\thesis\repos\data-science-ipython-notebooks
C:\Users\kenne\Documents\thesis\repos\DBDA-python
C:\Users\kenne\Documents\thesis\repos\digital-signal-processing-lecture
C:\Users\kenne\Documents\thesis\repos\elasticity
C:\Users\kenne\Documents\thesis\repos\EngComp1_offtheground
C:\Users\kenne\Documents\thesis\repos\EngComp2_takeoff
C:\Users\kenne\Documents\thesis\repos\EngComp3_tourdynamics
C:\Users\kenne\Documents\thesis\repos\ETL_with_Python
C:\Users\kenne\Documents\thesis\repos\fecon235
C:\Use

,cell_id,file_cell,file,nbformat,folder,repo,location,string,lines,cell_type,output,output_type
0,0,0.0,segregation-analysis.ipynb,3.0,notebooks,2014-08-st-louis-county-segregation,C:\Users\kenne\Documents\thesis\repos\2014-08-...,"[import pandas as pd, import numpy as np, pd.s...",3.0,code,empty,empty
1,1,1.0,segregation-analysis.ipynb,3.0,notebooks,2014-08-st-louis-county-segregation,C:\Users\kenne\Documents\thesis\repos\2014-08-...,[## The Data],1.0,markdown,empty,empty
2,2,2.0,segregation-analysis.ipynb,3.0,notebooks,2014-08-st-louis-county-segregation,C:\Users\kenne\Documents\thesis\repos\2014-08-...,[The data for this analysis comes from the Ame...,1.0,markdown,empty,empty
3,3,3.0,segregation-analysis.ipynb,3.0,notebooks,2014-08-st-louis-county-segregation,C:\Users\kenne\Documents\thesis\repos\2014-08-...,"[def read_acs_file(path, coldict):, return pd....",4.0,code,empty,empty
4,4,4.0,segregation-analysis.ipynb,3.0,notebooks,2014-08-st-louis-county-segregation,C:\Users\kenne\Documents\thesis\repos\2014-08-...,"[dp03_cols = {, ""Estimate; EMPLOYMENT STATUS -...",4.0,code,empty,empty
...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1031,0.0,pythreejs-trial.ipynb,4.0,_version_compatibility_checks,tutorial,C:\Users\kenne\Documents\thesis\repos\tutorial...,"[from pythreejs import *, import numpy as np, ...",5.0,code,empty,empty
1032,1032,1.0,pythreejs-trial.ipynb,4.0,_version_compatibility_checks,tutorial,C:\Users\kenne\Documents\thesis\repos\tutorial...,[ball = Mesh(geometry=SphereGeometry(radius=1)...,14.0,code,displayed data,display_data
1033,1033,2.0,pythreejs-trial.ipynb,4.0,_version_compatibility_checks,tutorial,C:\Users\kenne\Documents\thesis\repos\tutorial...,[ball.geometry.radius = 0.5],1.0,code,empty,empty
1034,1034,3.0,pythreejs-trial.ipynb,4.0,_version_compatibility_checks,tutorial,C:\Users\kenne\Documents\thesis\repos\tutorial...,"[# On windows, linewidth of the material has n...",16.0,code,displayed data,display_data


In [58]:
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST]) 


INDEX = "gallery_repos"
TYPE = "record"

In [59]:
for chuck in tqdm_notebook(split(gallery_repo_df, 500)):
    try:
        r = es.bulk(rec_to_actions(chuck))
    except:
        print('failed, skip this df')

In [60]:
!curl "http://localhost:9200/_cat/indices?v"

health status index         uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   restart       Gn2kwcjvRLehQoUX6WrTyQ   1   1     176812            0    916.6mb        916.6mb
yellow open   gallery_repos kNzSILR7T6Wp4RX1Cn9XaQ   1   1      42437            0     90.2mb         90.2mb
yellow open   gallery       x4vbbCLoT1aMRkZZLvVbOA   1   1       5654            0      7.6mb          7.6mb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   436  100   436    0     0  17440      0 --:--:-- --:--:-- --:--:-- 18166


### Put notebook collection into elastic search

In [5]:
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST],timeout=30,max_retries=10, retry_on_timeout=True) 


INDEX = "restart"
TYPE = "record"

In [6]:
sample_df = create_SE_from_folder('sample',es,'notebooks',0)

failed for file: nb_1002676.ipynb
failed for file: nb_1005326.ipynb
failed for file: nb_1007250.ipynb
failed for file: nb_1009478.ipynb
failed for file: nb_1009478.ipynb
failed for file: nb_1009478.ipynb
failed for file: nb_1009479.ipynb
failed for file: nb_1011216.ipynb
failed for file: nb_1012486.ipynb
failed for file: nb_1012605.ipynb
failed for file: nb_1014564.ipynb
failed for file: nb_1014564.ipynb
failed for file: nb_1016480.ipynb
failed for file: nb_1016480.ipynb
failed for file: nb_1017432.ipynb
failed for file: nb_1017432.ipynb
failed for file: nb_1017432.ipynb
failed for file: nb_1017432.ipynb
failed for file: nb_1017432.ipynb
failed for file: nb_1017552.ipynb
failed for file: nb_1017688.ipynb
failed for file: nb_1017688.ipynb
failed for file: nb_1017688.ipynb
failed for file: nb_1017688.ipynb
failed for file: nb_1018009.ipynb
failed for file: nb_1018177.ipynb
failed for file: nb_1018469.ipynb
failed for file: nb_1018764.ipynb
failed for file: nb_1018957.ipynb
failed for fil

In [34]:
sample_df[-1:]['']

,cell_id,file_cell,file,nbformat,folder,repo,location,string,lines,cell_type,output_type,output
177318,177318,13,nb_999948.ipynb,4,notebooks,sample,C:\Users\kenne\Documents\thesis\notebooks\nb_9...,"[%%time, y = x.dot(x.T).persist(), wait(y)]",3,code,empty,empty


In [ ]:
# sample_df.drop(columns=['char'])

In [ ]:
# set(sample_df.output_type)

sample_df[sample_df.output_type=='display_data']

In [9]:
for chuck in tqdm_notebook(split(sample_df, 500)):
    try:
        r = es.bulk(rec_to_actions(chuck))
    except:
        print('failed, skip this df') # UITELGGEN WAT ER MIS GING ZONDER DE TRY/EXCEPT > IN HET VERSLAG (224)
    
    
# for chuck in tqdm_notebook(split(sample_df.drop(columns=['char']), 10000)):
#     r = es.bulk(rec_to_actions(chuck))

failed, skip this df



In [10]:
# !curl -XDELETE "localhost:9200/sample_notebooks"
!curl "http://localhost:9200/_cat/indices?v"

health status index             uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   test_df_extra     DCUnYCeMSeSpqyf4guzzhw   1   1       6029            0    514.8mb        514.8mb
yellow open   restart           Gn2kwcjvRLehQoUX6WrTyQ   1   1     112498            0    898.7mb        898.7mb
yellow open   sample_notebooks1 kYo_GDRQTrmTrrxSklQDSQ   1   1     176812            0    915.7mb        915.7mb
yellow open   sample_notebooks  64SAHKxJSC6IqaDLYCuJ3A   1   1     176312            0    691.5mb        691.5mb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   565  100   565    0     0   8968      0 --:--:-- --:--:-- --:--:--  8828
100   565  100   565    0     0   8968      0 --:--:-- --:--:-- --:--:--  8828
